In [16]:
%run uniswap_trader.ipynb

In [17]:
class RetikTrader(TokenTrader):
    def __init__(self, rpc_url, wallet_address, private_key, token_address, 
                 usdt_address, eth_address, price_thresholds, lot_size_tokens, 
                 min_sell_amount_tokens, etherscan_api_key, contract_address, presale_id):
        
        super().__init__(rpc_url, wallet_address, private_key, token_address, 
                         usdt_address, eth_address, price_thresholds, lot_size_tokens, 
                         min_sell_amount_tokens, etherscan_api_key)

        self.contract_address = contract_address
        self.presale_id = presale_id
        self.contract = self.web3.eth.contract(address=Web3.to_checksum_address(self.contract_address), abi=self.get_contract_abi())

    def get_contract_abi(self):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={self.contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
        
    def claim_tokens(self):
        claim_txn = self.contract.functions.claimAmount(self.presale_id).buildTransaction({
            'from': self.wallet_address,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address)
        })
        signed_txn = self.web3.eth.account.sign_transaction(claim_txn, private_key=self.private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Tokens reclamados con éxito, transacción: {receipt.transactionHash.hex()}")
    
    def claimable_tokens(self):
        # Obtener datos desde blockchain
        user_data = self.contract.functions.userClaimData(self.wallet_address, self.presale_id).call()
        vesting_data = self.contract.functions.vesting(self.presale_id).call()
        # Estructurar los datos que se necesitan
        current_time = self.web3.eth.get_block('latest').timestamp
        start_time = vesting_data[0]
        initial_percent = vesting_data[1]
        vesting_time = vesting_data[2]
        vesting_percent = vesting_data[3]
        total_cycles = vesting_data[4]
        total_tokens = user_data[2]
        # Condiciones para continuar con el cálculo
        if not (vesting_time > 0):
            return 0
        if current_time < start_time:
            return 0
        # Cálculo de los tokens reclamables según tiempo
        elapsed_time = current_time - start_time
        cycles_completed = min(elapsed_time // vesting_time, total_cycles)
        initially_claimable = total_tokens * initial_percent / 1000
        additionally_claimable = (total_tokens * vesting_percent / 1000) * cycles_completed
        claimable_now = initially_claimable + additionally_claimable - user_data[4]
        return max(0, claimable_now)
    
    def manage_sales(self, claimable, balance, price_in_usdt):
        # Determinar cantidad total a vender basada en los umbrales de precios y lotes pendientes
        total_to_sell = 0
        lots_to_sell = []
        # Revisar cada lote y acumular la cantidad que puede venderse
        for i in range(len(self.price_thresholds)):
            if price_in_usdt >= self.price_thresholds[i] and self.sold_tokens[i] < self.lot_size:
                possible_sell = self.lot_size - self.sold_tokens[i]
                if possible_sell + total_to_sell <= balance + claimable:
                    total_to_sell += possible_sell
                    lots_to_sell.append((i, possible_sell))
                else:
                    break
        # Proceder a vender si hay algo que vender y la cantidad es adecuada
        if total_to_sell >= self.min_sell_amount:
            if claimable > 0 and claimable + balance >= total_to_sell:
                self.claim_tokens()
            print(f"Vendiendo {(total_to_sell / 10**self.token_decimals)} tokens a {price_in_usdt} USDT")
            tx_receipt = self.sell_token_for_base(self.token_address, self.eth_address, total_to_sell)
            print(f"Transacción confirmada: {tx_receipt}")
            
            # Actualizar estado según lo vendido
            for lot_index, amount in lots_to_sell:
                self.sold_tokens[lot_index] += amount
            
            # Guardar estado después de la venta
            self.save_state()

        # Si no se alcanza el monto mínimo para vender, esperar al siguiente ciclo
        elif total_to_sell > 0:
            print("Cantidad acumulada no suficiente para venta, esperando más acumulación o aumento de precio.")

    def trade(self):
        while sum(self.sold_tokens) < len(self.price_thresholds) * self.lot_size:
            try:
                claimable = int(self.claimable_tokens())
                print(f"Tokens reclamables: {(claimable / 10**self.token_decimals)} tokens")
                balance = int(self.get_token_balance(self.token_address))
                print(f"Balance actual: {(balance / 10**self.token_decimals)} tokens")
                price_in_usdt = self.get_token_price_in_usdt()
                print(f"Precio actual: {price_in_usdt} USDT")
                if price_in_usdt is not None and balance > self.min_sell_amount:
                    self.manage_sales(claimable, balance, price_in_usdt)
                else:
                    time.sleep(5)
                    continue
                # Espero 1 segundo
                time.sleep(1)
            except Exception as e:
                print(f"Ha ocurrido un error: {e}")
                time.sleep(1)
    

In [18]:
if __name__ == "__main__":
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    rpc_url = "https://rpc.ankr.com/eth" 
    wallet_address = os.getenv("WALLET_ADDRESS")
    private_key = os.getenv("PRIVATE_KEY")
    etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
    eth_address = "0x0000000000000000000000000000000000000000"
    usdt_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    token_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
    token_contract = "0x602C90D796D746b97a36f075d9f3b2892B9B07c2"
    presale_id = 2
    price_thresholds = np.linspace(0.49, 1.99, 20)
    lot_size_tokens = 2547
    min_sell_amount_tokens = 1000

    trader = RetikTrader(rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds,
                         lot_size_tokens, min_sell_amount_tokens, etherscan_api_key, token_contract, presale_id)
    trader.trade()

Tokens reclamables: 0.0 tokens
Balance actual: 0.0 tokens
Ha ocurrido un error al chequear el precio: ('execution reverted', 'no data')
Precio actual: None USDT
Tokens reclamables: 0.0 tokens
Balance actual: 0.0 tokens
Ha ocurrido un error al chequear el precio: ('execution reverted', 'no data')
Precio actual: None USDT


KeyboardInterrupt: 